In [1]:
import pandas as pd

In [2]:
companies_df = pd.read_csv('../data/S&P500.csv')
companies_df

,Unnamed: 0,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
498,498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [ ]:
companies_df['Symbol']

# check tickers: check xem co data tren yfinance ko
# neu ko co hoac data missing nhieu thi drop

# information quan trong: Security, GICS Sector, Sub industry -> merged vs yfinance data -> s3 bucket -> dashboard

# Function sau phai viet:
# 1. Function de read data vao s3 bucket
# 2. Function de automate data dung Lambda
# 3. Function de transform/clean data: ready for using -> m co the sua lai cai function cua t
#       - Handle neu ko tim dc ticker -> bo no ra khoi list ticker can download
#       - Handle missing value -> neu missing nhieu thi drop ticker day


0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [ ]:
import pandas as pd 
import os

def extract_companies():
    """
    fetch_stock_data:
    - Parameters: None
    - Output: Save a list of S&P500 companies as a csv file
    """

    filename = '../data/S&P500.csv'

    # Check if file exists
    if not os.path.exists(filename):
        table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        df = table[0]
        df.to_csv(filename)
        print(f"Successfully save S&P500 data into {filename}")
        return None
    
    print("File already exist. Skip downloading data!")
    return None

if __name__ == "__main__":
    extract_companies()

In [5]:
# ============================================================
# fetch_stock_data.py
#
# Python script to fetch data from Yahoo Finance and store the
# data locally.
# ============================================================

# import packages
import yfinance as yf
from datetime import datetime, timedelta
import os

def fetch_stock_data(tickers, start_date, end_date):
    """
    fetch_stock_data:
    - Parameters:
        + tickers: lists of stocks
        + start_date: the start date of the data
        + end_date: the end date of the data
    - Output: the stocks data given the start and end date for the list of stocks
    """

    print(f"Fetching data for: {tickers} ...")

    try: 
        # Specify the file name
        filename = f"../data/stock_data_{start_date}_to_{end_date}.csv"

        # Check if file exists
        if os.path.exists(filename):
            print(f"{filename} already exists. Skipping save.")
            return filename

        # Check if input is valid
        if not tickers or not isinstance(tickers, list):
            raise ValueError("Ticker list must be a non-empty list of symbols.")
        
        data = yf.download(tickers, start = start_date, end = end_date)

        # Check if data is successfully downloaded
        if data.empty:
            raise ValueError("No data returned. Please check the input again!")
        
        # Transform/Clean data
        data = data.stack(level = 0).rename_axis(['Date', 'Ticker']).reset_index()
        data = data.rename(columns={'Ticker':'Type'})

        # Save file locally
        data.to_csv(filename, index = False)

        print(f"Sucessfully saved data to {filename}")
        return filename
    
    except ValueError as er:
        print(f"Value Error: {er}")
    except Exception as e:
        print(f"Error occured during fetch/save: {e}")

    return None

if __name__ == "__main__":
    end_date = datetime.today() # Today’s date
    start_date = end_date - timedelta(days = 30 * 5)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    fetch_stock_data(["AAPL", "MSFT", "GOOGL"], start_date, end_date)

Fetching data for: ['AAPL', 'MSFT', 'GOOGL'] ...


C:\Users\tienv\AppData\Local\Temp\ipykernel_14728\1648551669.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start = start_date, end = end_date)
[*********************100%***********************]  3 of 3 completed

Sucessfully saved data to ../data/stock_data_2025-01-28_to_2025-06-27.csv



C:\Users\tienv\AppData\Local\Temp\ipykernel_14728\1648551669.py:45: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level = 0).rename_axis(['Date', 'Ticker']).reset_index()


## Direct Call After Cleaning

In [42]:
def get_sp500_tickers(filename = '../data/S&P500.csv'):
    try:
        df = pd.read_csv(filename)

        # Ensure 'Symbol' column exists
        if 'Symbol' not in df.columns:
            raise ValueError("CSV file must contain a 'Symbol' column.")
        
        # Drop missing or blank values
        tickers = df['Symbol'].dropna().astype(str).str.strip()
        tickers = tickers[tickers != ''].tolist()

        # Replace '.' with '-' for Yahoo Finance format
        cleaned_tickers = [ticker.replace('.', '-') for ticker in tickers]

        # Final sanity check: no empty strings
        cleaned_tickers = [t for t in cleaned_tickers if t]


        return cleaned_tickers
    except Exception as e:
        print(f"Error in get_sp500_tickers: {e}")
        return []  

if __name__ == "__main__":
    cleaned_tickers = get_sp500_tickers()
    end_date = datetime.today() # Today’s date
    start_date = end_date - timedelta(days = 30 * 5)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    fetch_stock_data(cleaned_tickers, start_date, end_date)

Fetching data for: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DH